In [ ]:
import subprocess
def export_firewall_settings(file):
    if " " in file and not (file[0] == file[1] == '"':
        file = '"%s"'%file
    cmd = "powershell -command \"Get-NetFirewallRule | export-csv %s -NoTypeInformation\"" % file
    subprocess.call(cmd)

In [ ]:
def lines(csv):
    with open(csv, 'r') as f:
        lines = [[s.strip('"') for s in l.split(",")] for l in f.read().splitlines()]
    return sorted(lines)
def compare_firewall_settings(ref, tmpfile):
    rlines = lines(ref)
    tlines = lines(tmpfile)
    return rlines == tlines